# Callbacks

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cbouy/mols2grid/blob/master/docs/notebooks/callbacks.ipynb)

In [ ]:
# uncomment and run if you're on Google Colab
# !pip install rdkit-pypi mols2grid py3Dmol
# !wget https://raw.githubusercontent.com/rdkit/rdkit/master/Docs/Book/data/solubility.test.sdf

In [ ]:
import mols2grid
from pathlib import Path
from rdkit import RDConfig
from ipywidgets import widgets
import urllib.request, urllib.parse
from urllib.error import HTTPError
from IPython.display import display
import py3Dmol
SDF_FILE = (f"{RDConfig.RDDocsDir}/Book/data/solubility.test.sdf"
            if Path(RDConfig.RDDocsDir).is_dir() else "solubility.test.sdf")

Callbacks are **functions that are executed when you click on a molecule's image**. They can be written in *JavaScript* or *Python*.

It can be used to display some additional information on the molecule or run some more complex code like database queries, docking or machine-learning predictions.

## Python

For Python callbacks, you need to declare a function that takes a dictionnary as first argument. This dictionnary contains all the data related to the molecule you've just clicked on. All the data fields are **parsed as strings**, except for the index, "mols2grid-id", which is always parsed as an integer.

For example, the SMILES of the molecule will be available as `data["SMILES"]`. If the field contains spaces, they will be converted to hyphens, *i.e.* a field called `mol weight` will be available as `data["mol-weight"]`.

Beware, **Python callbacks can only run on classical Jupyter-notebooks and Google Colab, not JupyterLab or VSCode!**

Also, using print or any other "output" functions inside the callback will not display anything by default. You need to use ipywidgets's `Output` widget to capture what the function is trying to display, and then show it.

### Basic print example

In this simple example, we'll just show the content of the data dictionnary.

In [ ]:
output = widgets.Output()
# the Output widget let's us capture the output generated by the callback function
# its presence is mandatory if you want to print/display some info with your callback
@output.capture(clear_output=True, wait=True)
def show_data(data):
    data.pop("img")
    print(data)

view = mols2grid.display(
    SDF_FILE, mol_col="mol",
    tooltip_trigger="hover",
    callback=show_data,
)
display(view)
output

We can also make more complex operations with callbacks.

### Displaying the 3D structure with py3Dmol

Here, we'll query PubChem for the molecule based on its SMILES, then fetch the 3D structure and display it with py3Dmol.

In [ ]:
output = widgets.Output()
@output.capture(clear_output=True, wait=True)
def show_3d(data):
    """Query PubChem to download the SDFile with 3D coordinates and
    display the molecule with py3Dmol
    """
    url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/{}/SDF?record_type=3d"
    smi = urllib.parse.quote(data["SMILES"])
    try:
        response = urllib.request.urlopen(url.format(smi))
    except HTTPError:
        print(f"Could not find corresponding match on PubChem")
        print(data["SMILES"])
    else:
        sdf = response.read().decode()
        view = py3Dmol.view(height=300, width=800)
        view.addModel(sdf, "sdf")
        view.setStyle({'stick': {}})
        view.zoomTo()
        view.show()

view = mols2grid.display(
    SDF_FILE, mol_col="mol",
    tooltip_trigger="hover",
    callback=show_3d,
)
display(view)
output

## JavaScript

We can also write JavaScript callbacks, which have the advantage to be able to run on almost any platform.

JS callbacks don't require to declare a function, and you can directly access and use the `data` object similarly to Python in your callback script.

### Basic JS example

In [ ]:
js_callback = """
// remove image from data
delete data["img"];
// convert data object to text
txt = JSON.stringify(data);
// show data in alert window
alert(txt);
"""

mols2grid.display(
    SDF_FILE, mol_col="mol",
    tooltip_trigger="hover",
    callback=js_callback,
)

To display fancy popup windows on click, a helper function is available: `mols2grid.make_popup_callback`.

It requires a title as well as some html code to format and display the information that you'd like to show. All of the values inside the data object can be inserted in the `title` and `html` arguments using `${data["field_name"]}`. Additionally, you can execute a prerequisite JavaScript snippet to create variables that are then also accessible in the html code.

### Display a popup containing descriptors

In the following exemple, we create an RDKit molecule using the SMILES of the molecule (the SMILES field is always present in the data object, no matter your input when creating the grid).

We then create a larger SVG image of the molecule, and calculate some descriptors.

Finally, we inject these variables inside the HTML code. You can also style the popup window through the style argument.

In [ ]:
js_callback = mols2grid.make_popup_callback(
    title="${data['NAME']}",
    js="""
        var mol = RDKit.get_mol(data["SMILES"]);
        var svg = mol.get_svg(400, 300);
        var desc = JSON.parse(mol.get_descriptors());
        mol.delete();
    """,
    html="""
        <div class="row">
          <div class="col">${svg}</div>
          <div class="col">
            <b>Molecular weight</b>: ${desc.amw}<br/>
            <b>HBond Acceptors</b>: ${desc.NumHBA}<br/>
            <b>HBond Donors</b>: ${desc.NumHBD}<br/>
            <b>ClogP</b>: ${desc.CrippenClogP}<br/>
          </div>
        </div>""",
    style="max-width: 80%;",
)

mols2grid.display(
    SDF_FILE, mol_col="mol",
    tooltip_trigger="hover",
    callback=js_callback,
)

It is possible to load additional JS libraries by passing `custom_header="<script src=...></script>"` to `mols2grid.display`, and they will then be available in the callback.

### Displaying the 3D structure with 3Dmol.js

In the following example, we query PubChem using the SMILES of the molecule, then fetch the 3D structure and display it the SDFile with 3Dmol.js:

In [ ]:
# load 3Dmol.js in header
custom_header = """
<script src="https://cdnjs.cloudflare.com/ajax/libs/3Dmol/1.8.0/3Dmol-nojquery-min.js" integrity="sha512-9iiTgstim185ZZPL/nZ+t+MLMmIbZEMfoZ1swSBUhxt4AukOPY34yyO2217X1dN5ziVMKi+YLmp/JBj+KyEaUQ==" crossorigin="anonymous" referrerpolicy="no-referrer"></script>
"""

js_callback = mols2grid.make_popup_callback(
    title="${data['NAME']}",
    js="""
    // prepare PubChem query to fetch 3D SDF from SMILES
    let base_url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/';
    let escaped_smi = encodeURIComponent(data['SMILES']);
    var sdf_url = base_url + escaped_smi + '/SDF?record_type=3d';

    // fetch file and display with 3Dmol.js
    $(document).ready(function() {
        let element = $('#molviewer');
        let config = { backgroundColor: 'white' };
        let viewer = $3Dmol.createViewer(element, config);
        $.ajax(sdf_url, { 
            success: function(data) {
                viewer.addModel(data, "sdf");
                viewer.setStyle({}, {stick: {}});
                viewer.zoomTo();
                viewer.render();
            },
            error: function(hdr, status, err) {
                console.error("Failed to load SDF " + sdf_url + ": " + err);
            },
        });
    });
    """,
    html='<div id="molviewer" style="width: 100%; height: 350px;"></div>',
    style="max-width: 80%",
)

mols2grid.display(
    SDF_FILE, mol_col="mol",
    tooltip_trigger="hover",
    callback=js_callback,
    custom_header=custom_header,
)